In [52]:
import bptf
from bptf import BPTF
import numpy as np
import pandas as pd
import sparse
import os
import shutil
from tqdm import tqdm
import pickle
from concurrent.futures import ProcessPoolExecutor, as_completed
import scipy.stats as st
import matplotlib.pyplot as plt
import torch
import tensorly
import cupy
import multiprocessing
from joblib import Parallel, delayed
from tqdm.contrib.concurrent import process_map

os.getcwd()

'c:\\Users\\luiyu\\OneDrive - The University of Chicago\\UChicago_MS_Stat\\aaron_schein\\bptf_new'

# Utility functions

In [53]:
def dataframe_to_sparse_tensor(data, country_indices, date_indices, cameo_col='CAMEO_Code', events_col='Num_Events'):
    """
    Converts a pandas DataFrame into an sptensor.
    
    Parameters:
        data (pd.DataFrame): The DataFrame containing the data.
        country_indices (pd.DataFrame): DataFrame with country indices.
        date_indices (pd.DataFrame): DataFrame with date indices.
        cameo_col (str): Name of the CAMEO code column in 'data'. Defaults to 'CAMEO_Code'.
        events_col (str): Name of the column containing event counts in 'data'. Defaults to 'Num_Events'.
    
    Returns:
        sptensor: The resulting sparse tensor.
    """
    
    # Define the shape of the tensor (V, V, A, T)
    V = len(country_indices)
    A = 20  # Assuming the CAMEO code has 20 distinct values
    T = len(date_indices)
    
    shape = (V, V, A, T)
    
    # Initialize empty subs and vals
    subs = ([], [], [], [])
    vals = []

    # Iterate through the DataFrame to populate the tensor
    for i in range(len(data)):

        source_country = country_indices.loc[country_indices['country'] == data['Source_Country_Code'].iloc[i]]
        source_country_index = int(source_country.iloc[0, 1])

        target_country = country_indices.loc[country_indices['country'] == data['Target_Country_Code'].iloc[i]]
        target_country_index = int(target_country.iloc[0, 1])

        action_index = int(data[cameo_col].iloc[i] - 1)  # Adjust CAMEO code to 0-based index

        date = date_indices.loc[date_indices['date'] == data['formatteddate'].iloc[i]]
        date_index = int(date.iloc[0, 1])

        # Append indices and values
        subs[0].append(source_country_index)
        subs[1].append(target_country_index)
        subs[2].append(action_index)
        subs[3].append(date_index)
        vals = np.append(vals, data[events_col].iloc[i])

    # Convert subs to a tuple of numpy arrays and vals to a numpy array
    subs = tuple(np.array(s, dtype=int) for s in subs)
    vals = np.array(vals)

    # Create the sparse COO tensor
    Y = sparse.COO(coords=subs, data=vals, shape=shape)
    
    return Y

def add_sparse_tensors(tensor1, tensor2):
    assert tensor1.shape == tensor2.shape, "Tensors must have the same shape to be added."
    
    # Directly add the two tensors
    result_tensor = tensor1 + tensor2
    
    return result_tensor


def sum_sparse_tensor_list(tensor_list):
    if not tensor_list:
        raise ValueError("The list of tensors is empty.")
    
    # Use sum with a starting tensor of 0 in the shape of the tensors
    result_tensor = sum(tensor_list[1:], start=tensor_list[0])
    
    return result_tensor

# Get and process data

In [54]:
# read files by batch
folder = "\\dyadic_data_2000_2020_inclusive\\"
data_filepath = os.getcwd() + folder
files = os.listdir(data_filepath)
for filepath in tqdm(files, desc = 'Reading data'):
    complete_filepath = data_filepath + filepath
    file = pd.read_csv(complete_filepath)
    if filepath == files[0]:
        data = file
    else:
        data = pd.concat([data, file])
    del file
data = data.sort_values(by='Num_Events', ascending=False)

# filter by date to get a smaller dataset to play around with
# Also my laptop can only fit a 10/11 year dataset
years = [str(2000 + x) for x in range(0, 13)]
data = data[data['formatteddate'].str.startswith(tuple(years))]
# Take only ICEWS data
data = data[data['Database'] == 'ICEWS']

# collapse by month
data['formatteddate'] = data['formatteddate'].str[:7]
data = data.groupby(['Source_Country_Code', 'Target_Country_Code', 'CAMEO_Code', 'formatteddate', 'Database'])
data = data.sum().reset_index()

# get unique list of countries
countries = pd.concat([data['Source_Country_Code'], data['Target_Country_Code']])
countries = pd.unique(countries)
country_indices = pd.DataFrame({
    'country' : countries,
    'index' : range(len(countries))
})
del countries

# there's no need to get a list of actions since it's just 1 to 20

# get unique list of dates
# Make sure to change the date format and frequency of the date range if you change the time unit
date_indices = pd.date_range(
    start=pd.to_datetime(data['formatteddate'], format='%Y-%m').min().strftime('%Y-%m'), 
    end=pd.to_datetime(data['formatteddate'], format='%Y-%m').max().strftime('%Y-%m'),
    freq='MS')
date_indices = date_indices.strftime('%Y-%m').to_list()
date_indices = pd.DataFrame({
    'date' : date_indices,
    'index' : range(len(date_indices))
})
date_indices['date'] = date_indices['date'].str[:7]

Reading data: 100%|██████████| 85/85 [00:55<00:00,  1.52it/s]


# Convert dataframe to sparse tensor

In [55]:
num_of_batches = 10000
data = np.array_split(data, num_of_batches)

c:\Users\luiyu\anaconda3\envs\py_3_11\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [56]:
data[0].shape

(150, 6)

In [57]:
# Y = []
# progressbar = tqdm(range(len(data)))
# for batch in progressbar:
#     progressbar.set_description(f'Now convert batch {batch} out of total {len(data)} batches.')
#     Y.append(dataframe_to_sparse_tensor(data[batch], country_indices, date_indices))
# Y = sum(Y)
# del data

In [58]:
num_cores = multiprocessing.cpu_count()

if __name__ == '__main__':
    data = Parallel(n_jobs = num_cores)(
        delayed(dataframe_to_sparse_tensor)(batch, country_indices, date_indices)
        for batch in tqdm(data, desc='Converting to sparse tensors')
    )
# Y = sparse.COO(coords=np.empty((len(data[0].shape), 0), dtype=int), data=np.array([]), shape=data[0].shape)
# for batch in tqdm(data):
#     Y += batch

Converting to sparse tensors: 100%|██████████| 10000/10000 [02:13<00:00, 74.64it/s]


In [59]:
n = 10
while len(data) > 10:
    data = [data[i:i + n] for i in range(0, len(data), n)]
    data = [sum(batch) for batch in data]
data = sum(data)

In [61]:
n_components = 100
bptf_ICEWS = BPTF(data_shape=data.shape, n_components=n_components)
bptf_ICEWS.fit(data, max_iter = 100, verbose = True)
for j in range(len(data.shape)):
    assert bptf_ICEWS.G_DK_M[j].shape == (data.shape[j], n_components)

ITERATION 0:	Time: 0.000000	Objective: -19949045278.59	Change: nan	


  1%|          | 1/100 [00:23<38:57, 23.61s/it]

ITERATION 1:	Time: 23.611221	Objective: -10026732.82	Change: 9.99497e-01	


  2%|▏         | 2/100 [00:46<38:05, 23.33s/it]

ITERATION 2:	Time: 23.124402	Objective: -9625551.53	Change: 4.00112e-02	


...

ITERATION 94:	Time: 22.516852	Objective: 5145728.63	Change: 1.27739e-04	


 95%|█████████▌| 95/100 [36:39<01:52, 22.55s/it]

ITERATION 95:	Time: 22.577482	Objective: 5146261.29	Change: 1.03515e-04	


 95%|█████████▌| 95/100 [37:01<01:56, 23.39s/it]

ITERATION 96:	Time: 22.362030	Objective: 5146714.30	Change: 8.80275e-05	


In [62]:
cp_init = tensorly.cp_tensor.CPTensor(
        tensorly.decomposition._cp.initialize_cp(
            data, non_negative = True, init = 'random', rank = n_components
            )
        )
tensor_mu, _ = tensorly.decomposition.non_negative_parafac(
    data, rank=n_components, init=cp_init, return_errors=True
    )

In [67]:
assert data.todense().shape == bptf_ICEWS.reconstruct(drop_diag=True, style='geometric').shape

In [78]:
bptf_error = torch.norm(
    torch.from_numpy(data.todense()) - torch.from_numpy(bptf_ICEWS.reconstruct(drop_diag=True, style='post_pred', n_samples=100)), 'fro'
    ) / torch.norm(
        torch.from_numpy(data.todense()), 'fro'
        )
nonneg_cp = torch.norm(torch.from_numpy(data.todense()) - torch.from_numpy(tensorly.cp_to_tensor(tensor_mu)), 'fro') / torch.norm(torch.from_numpy(data.todense()), 'fro')

sampling: 100%|██████████| 100/100 [02:27<00:00,  1.47s/it]


In [79]:
print(f'BPTF error = {bptf_error}, Non-negative CP error = {nonneg_cp}')

BPTF error = 0.9594820676916082, Non-negative CP error = 0.3618835681327173
